# Music Autotagging Using AWS GPU 

This Project was build in AWS, it use local connection to the dataset, LSTM running on GPU

In [1]:
import pip
pip.main(['install', 'np_utils'])
pip.main(['install', 'theano'])
pip.main(['install', 'sklearn'])
pip.main(['install', 'tflearn'])
pip.main(['install', 'boto3'])
pip.main(['install', 'sqlite3'])
pip.main(['install', 'tables'])

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


  Could not find a version that satisfies the requirement sqlite3 (from versions: )
No matching distribution found for sqlite3
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


0

In [2]:
import np_utils
import theano
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
import keras as ks
import pandas as pd
import numpy as np

import hdf5_getters
from os import listdir

from itertools import product
from keras.optimizers import RMSprop
from keras.models import Sequential,load_model
from keras.layers import CuDNNLSTM, Dense,TimeDistributed,Dropout,CuDNNLSTM

from sklearn.metrics import confusion_matrix
import numpy as np
import time
from keras.optimizers import SGD
import tensorflow as tf
import keras.backend as K

import sklearn
import tflearn
from sklearn.metrics import roc_curve, auc
import boto3
import os
import tables
import sqlite3
import json
import ast

Using TensorFlow backend.
From /jet/var/python/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:198: retry (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
def get_file(path,vec_path,vec_id):
    files = listdir(path)
    for file_ in files:
        if len(file_.split('.'))==2:
            vec_path.append(path+'/'+file_)
            vec_id.append(file_)
        else:
            get_file(path+'/'+file_,vec_path,vec_id)
    return vec_path    

In [4]:
def get_tags(c,tid):
    tid =  tid.split('.')[0]
    sql = "SELECT tags.tag, tid_tag.val FROM tid_tag, tids, tags WHERE tags.ROWID=tid_tag.tag AND tid_tag.tid=tids.ROWID and tids.tid='%s'" % tid
    
    res = c.execute(sql)
    data = res.fetchall()
    
    return data

In [5]:
n_tags = 50
path = '/mnt/snap/data/'
tags = pd.read_table('./lastfm_top_tags.txt',sep='\t',nrows=n_tags,names=['tag','occurrence'])

In [6]:
'''vec_path = []
vec_id = []
%time vec_path = get_file(path,vec_path,vec_id)'''

'vec_path = []\nvec_id = []\n%time vec_path = get_file(path,vec_path,vec_id)'

In [7]:
import time

In [8]:
files_size=['80000','160000','240000','320000' ]

In [9]:
#This part of the code just need to run once to generate the files. This is done, because getting the data was slow 
#and to expensive to rerun everytime the kernel was initialized or restarted.
'''start = time.time()
count = 1
conn = sqlite3.connect('lastfm_tags.db')
c = conn.cursor()
output = {}
music_size = {}
for file_,music_id in zip(vec_path,vec_id):
    
    
    music_tags  = get_tags(c,music_id)
    if len (music_tags)==0:
        continue
    
    s = set(np.transpose(music_tags )[0])
    intersect = s.intersection(tags['tag'])
    if len (intersect)==0:
        continue
   
    
    h5 = hdf5_getters.open_h5_file_read(file_)
    confidence = hdf5_getters.get_segments_confidence(h5)
    try:
        music_size[len(confidence)].append(file_)
        output[len(confidence)].append(music_tags)
        
    except:
        music_size[len(confidence)] = [file_]
        output[len(confidence)] = [music_tags]
    h5.close()
    
    
    if count%10000==0:
        print(count,time.time()-start)
    if count%80000==0:
        
        
 
        json1 = json.dumps(output)
        f = open("./data/out_dict"+str(count)+".json","w")
        f.write(json1)
        f.close()
        file_sizes = open('./data/size_dict'+str(count)+'.txt','w')
        keys = np.asarray(list(music_size.keys()))
        keys = np.sort(keys)
        for key in keys:
            value = music_size[key]
            value = str(value)
            file_sizes.write(str(key)+';'+value+'\n')
        file_sizes.close()
        output = {}
        music_size = {}
    count+=1
conn.close()    
json1 = json.dumps(output)
f = open("./data/out_dict"+str(count)+".json","w")
f.write(json1)
f.close()
file_sizes = open('./data/size_dict'+str(count)+'.txt','w')
keys = np.asarray(list(music_size.keys()))
keys = np.sort(keys)
for key in keys:
    value = music_size[key]
    value = str(value)
    file_sizes.write(str(key)+';'+value+'\n')
file_sizes.close()
output = {}
music_size = {}'''


'start = time.time()\ncount = 1\nconn = sqlite3.connect(\'lastfm_tags.db\')\nc = conn.cursor()\noutput = {}\nmusic_size = {}\nfor file_,music_id in zip(vec_path,vec_id):\n    \n    \n    music_tags  = get_tags(c,music_id)\n    if len (music_tags)==0:\n        continue\n    \n    s = set(np.transpose(music_tags )[0])\n    intersect = s.intersection(tags[\'tag\'])\n    if len (intersect)==0:\n        continue\n   \n    \n    h5 = hdf5_getters.open_h5_file_read(file_)\n    confidence = hdf5_getters.get_segments_confidence(h5)\n    try:\n        music_size[len(confidence)].append(file_)\n        output[len(confidence)].append(music_tags)\n        \n    except:\n        music_size[len(confidence)] = [file_]\n        output[len(confidence)] = [music_tags]\n    h5.close()\n    #confidence_vec.append( confidence)\n    #loudness_max = ( hdf5_getters.get_segments_loudness_max(h5))\n    #max_time = (hdf5_getters.get_segments_loudness_max_time(h5))\n    #loudness_star = (hdf5_getters.get_segments_

In [10]:
test_vec_file = open('./data/test_vec.txt','r')
test_vec = ast.literal_eval(test_vec_file.readlines()[0])


In [11]:
train_vec_file = open('./data/train_vec.txt','r')
train_path_fixed = ast.literal_eval(train_vec_file.readlines()[0])

In [12]:
def roc_auc_score(y_true,y_pred):
    """ ROC AUC Score.
    Approximates the Area Under Curve score, using approximation based on
    the Wilcoxon-Mann-Whitney U statistic.
    Yan, L., Dodier, R., Mozer, M. C., & Wolniewicz, R. (2003).
    Optimizing Classifier Performance via an Approximation to the Wilcoxon-Mann-Whitney Statistic.
    Measures overall performance for a full range of threshold levels.
    Arguments:
        y_pred: `Tensor`. Predicted values.
        y_true: `Tensor` . Targets (labels), a probability distribution.
    """
    
    with tf.name_scope("RocAucScore"):
    
        pos = tf.boolean_mask(y_pred, tf.cast(y_true, tf.bool))
        neg = tf.boolean_mask(y_pred, ~tf.cast(y_true, tf.bool))

        pos = tf.expand_dims(pos, 0)
        neg = tf.expand_dims(neg, 1)

        # original paper suggests performance is robust to exact parameter choice
        gamma = 0.2
        p     = 3

        difference = tf.zeros_like(pos * neg) + pos - neg - gamma

        masked = tf.boolean_mask(difference, difference < 0.0)

    return tf.reduce_sum(tf.pow(-masked, p)) 

In [13]:
def mine_auc(y_true,y_pred):
    return tflearn.objectives.roc_auc_score(y_pred, y_true)

In [14]:
def init_model():
# create model
    
    model = Sequential()
    model.add(CuDNNLSTM(32, return_sequences=False, input_shape=(None, 27)))
    #model.add(CuDNNLSTM(32, return_sequences=False,))
    #model.add(CuDNNLSTM(32, return_sequences=True,))
    #model.add(CuDNNLSTM(32, return_sequences=True,))
    #model.add(CuDNNLSTM(32, return_sequences=False,))
    #model.add(Dropout(0.5))
    model.add((Dense(n_tags, activation='sigmoid')))
    #optimizer = RMSprop(lr=0.0000001)
    sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
    model.compile(loss=mine_auc,
                      optimizer=sgd,
                      metrics=['accuracy'])
    return model

In [15]:

def train_model(model,epochs,x_train,y_train):
    
    #model.fit(X_train, y_train callbacks=[roc_callback(training_data=(X_train, y_train),validation_data=(X_test, y_test))])
    model.fit(x_train, y_train, batch_size=1, epochs=epochs, shuffle=False, verbose=0)
    return model

In [16]:
def init_vecs():
    array1 = []
    array2 = []
    tuples = []
    loudness_vec = []
    confidence_vec = []
    loudness_max_vec = []
    max_time_vec = []
    loudness_start_vec = []
    pitches_vec = []
    segments_start_vec = []
    segments_timbre_vec = []
    y_train = []
    array_id =[]


    
    for i in range(11):
        segments_timbre_vec.append([])
        pitches_vec.append([])
    
    
    
    return y_train,array_id,array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec,max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec

In [17]:
def get_musics_informations(array_id,array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec, max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec,values):
    for file_ in values:
        
        h5 = hdf5_getters.open_h5_file_read(file_)
        confidence = hdf5_getters.get_segments_confidence(h5)
        
        confidence_vec.append( confidence)
        loudness_max_vec.append( hdf5_getters.get_segments_loudness_max(h5))
        max_time_vec.append(hdf5_getters.get_segments_loudness_max_time(h5))
        loudness_start_vec.append(hdf5_getters.get_segments_loudness_start(h5))

        segments_start_vec.append(hdf5_getters.get_segments_start(h5))

        for i in range(len(segments_timbre_vec)):
            segments_timbre_vec[i].append(np.transpose(hdf5_getters.get_segments_timbre(h5))[i])
            pitches_vec[i].append(np.transpose(hdf5_getters.get_segments_pitches(h5))[i])
        array_id.append([hdf5_getters.get_song_id(h5)])
        array1.append([hdf5_getters.get_song_id(h5)] *len(confidence))
        array2.append( range(len(confidence)))

        h5.close()
    return array_id,array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec, max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec


In [18]:
def generate_input_dataset(array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec, max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec):
    array1 = np.concatenate(array1)
    array2 = np.concatenate(array2)
    arrays = [array1,array2]
    tuples = list(zip(*arrays))
    index = pd.MultiIndex.from_tuples(tuples,names=['ID','Tempo'])
    pandas_input = [np.concatenate(confidence_vec),np.concatenate(loudness_max_vec),
                    np.concatenate(max_time_vec),np.concatenate(loudness_start_vec),

                    np.concatenate(segments_start_vec)]
    pandas_input = np.transpose(pandas_input)
    #pandas_input = np.concatenate(pandas_input,pitches_vec)
    #pandas_input = np.concatenate(pandas_input,segments_timbre)
    dataframe = pd.DataFrame(pandas_input,index=index,columns=['confidence_vec','loudness_max_vec','max_time_vec',
                                                               'loudness_start_vec','segments_start_vec'])
    for i in range(len(segments_timbre_vec)):
        segments_timbre_vec[i] = np.concatenate(segments_timbre_vec[i])
        pitches_vec[i] = np.concatenate(pitches_vec[i])
        name_timbre = 'timbre' + str(i)
        name_pitches = 'pitch'+ str(i)
        dataframe[name_timbre] = segments_timbre_vec[i]
        dataframe[name_pitches] = pitches_vec[i]
    
    
    return dataframe,index

In [19]:
def generate_output_dataset(tags,output,keys,values,index,array_id):
    dataframe_out = pd.DataFrame(index=index,columns=tags['tag'])
    dataframe_out=dataframe_out.fillna(value=0)
    for i in range(len(array_id)):
        music_tags = (set( np.transpose(output[str(keys)][i] )[0]).intersection(set(tags['tag'])))
        for tag_col in music_tags:
                dataframe_out[tag_col][array_id[i][0]]=1
    return dataframe_out

In [20]:
def to_np_array(dataframe):
    m,n = len(dataframe.index.levels[0]), len(dataframe.index.levels[1])
    arr = dataframe.values.reshape(m,n,-1)
    return arr

In [21]:
'''for size in files_size: 
    print (size)
    
    
    with open('./data/out_dict'+size+'.json') as handle:
        output = json.loads(handle.read())
    
    count = 0
    scores = []

    file_sizes = pd.read_csv('./data/size_dict'+size+'.txt',sep=';',names=['Key','Values'])
    file_sizes.index=file_sizes['Key']
    
    
    break
output'''

"for size in files_size: \n    print (size)\n    \n    \n    with open('./data/out_dict'+size+'.json') as handle:\n        output = json.loads(handle.read())\n    \n    count = 0\n    scores = []\n\n    file_sizes = pd.read_csv('./data/size_dict'+size+'.txt',sep=';',names=['Key','Values'])\n    file_sizes.index=file_sizes['Key']\n    \n    \n    break\noutput"

In [22]:

start_time = time.time()
model = init_model()
epochs = 1
#epochs_manual=10
pred_vec = []
real_vec = []
camadas = 1
try:
    for size in files_size: 
        print (size)


        with open('./data/out_dict'+size+'.json') as handle:
            output = json.loads(handle.read())

        count = 0
        scores = []

        file_sizes = open('./data/size_dict'+size+'.txt','r')

        #for keys in keys_vec:
        #    values = music_size[keys]
        for line in file_sizes.readlines():

            start_time1 = time.time()
            keys,values =  line.split(';')
            keys = int(keys)
            values = np.array(np.matrix(values)).ravel()
            intersect_vec = set(values)-set(train_path_fixed)
            index_test_vec=[]
            if  len(intersect_vec)>0:
                for intersect in intersect_vec:
                    index_test = (list(values).index(intersect))
                    index_test_vec.append(index_test)
                values=np.delete(values,index_test_vec)    
                output_vec = np.delete(output[str(keys)],index_test_vec)
                output[str(keys)] = output_vec
            if len(values)<1:
                continue
            y_train,array_id,array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec, max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec = init_vecs()
            array_id,array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec, max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec = get_musics_informations(array_id,array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec, max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec,values)

            dataframe,index = generate_input_dataset(array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec, max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec) 
            dataframe_out = generate_output_dataset(tags,output,keys,values,index,array_id)



            #dataframe = pd.read_csv('./../datacsv/%s.csv' % keys,index_col = ['ID','Tempo'])
            #dataframe_out = pd.read_csv('./../datacsv_out/%s.csv' % keys,index_col = ['ID','Tempo'])

            try:
                dataframe = to_np_array(dataframe)
                dataframe_out = to_np_array(dataframe_out)

            except:
                log_file = open('log_error_file_test6.txt','a')
                log_file.write(str(keys))
                log_file.write('\n')
                log_file.close()
                continue
            y_train = []
            for i in range(len(dataframe_out)):
                y_train.append(dataframe_out[i][0])


            y_train = np.asarray(y_train).reshape((-1, 50))

            model = train_model(model,epochs,dataframe,y_train)

            datatest = dataframe


            if count%25==0:
                log_file = open('log_file_25_test6.txt','a')
                log_file.write(str(keys)+','+str(time.time()-start_time))
                log_file.write('\n')
                log_file.close() 

            if count%150==0:
                print (size,count,(time.time()-start_time))

                #print y_pred
                #print 'Auc_score: %s' % (str(auc_value))
                #model.save('./models/model_auc_%s_l_%s_e_%s_r_%s' % ('2_32',epochs_manual,epochs,it_epochs)) 
            count +=1

            #array_id.append([hdf5_getters.get_song_id(h5)])
            #array1.append([hdf5_getters.get_song_id(h5)] *len(confidence))
            #array2.append( range(len(confidence)))

        model.save('./models/model_auc_%s_%s_%s' % (camadas,epochs,size)) 

        '''print (sklearn.metrics.roc_auc_score(np.asarray(real_vec), np.asarray(pred_vec),average='macro'))
        print (sklearn.metrics.roc_auc_score(np.asarray(real_vec), np.asarray(pred_vec),average='micro'))
        print (sklearn.metrics.roc_auc_score(np.asarray(real_vec), np.asarray(pred_vec),average='samples'))'''
except:
    print('error')
print(time.time()-start_time)
model.save('./models/model_auc_%s_%s_%s_final' % (camadas,epochs,size))

From /jet/var/python/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1247: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
From /jet/var/python/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1349: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


80000


/jet/var/python/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


80000 0 6.477267265319824
80000 150 48.18399143218994
80000 300 129.14771485328674
80000 450 375.61834502220154
80000 600 975.282347202301
80000 750 1775.413353919983
80000 900 2500.4504895210266
80000 1050 3013.867310523987
80000 1200 3380.584093809128
80000 1350 3630.752851486206
80000 1500 3820.9651849269867
80000 1650 3965.1844251155853
80000 1800 4080.9153106212616
80000 1950 4181.417778491974
80000 2100 4268.53727722168
80000 2250 4346.319153308868
80000 2400 4413.283097743988
80000 2550 4482.013801574707
80000 2700 4539.99022769928
80000 2850 4594.125570297241
80000 3000 4642.781368255615
80000 3150 4691.260207653046
160000
160000 0 4731.106090784073
160000 150 4773.109721183777
160000 300 4852.9436547756195
160000 450 5086.456908941269
160000 600 5693.691328048706
160000 750 6494.594488143921
160000 900 7213.0033440589905
160000 1050 7743.870124578476
160000 1200 8110.182434558868
160000 1350 8362.764414310455
160000 1500 8549.368165016174
160000 1650 8693.638087511063
160000 1

In [23]:
test_files = ['_test']
model.save('./models/model_auc_%s_%s_%s_final' % (camadas,epochs,size))
#model = load_model('./models/model_auc_primeiro',custom_objects={'mine_auc': mine_auc})

In [24]:
start_time = time.time()

#epochs_manual=10
pred_vec = []
real_vec = []
for size in test_files: 
    print (size)
    
    
    with open('./data/out_dict'+size+'.json') as handle:
        output = json.loads(handle.read())
    
    count = 0
    scores = []

    file_sizes = open('./data/size_dict'+size+'.txt','r')
    

    for line in file_sizes.readlines():
        
        start_time1 = time.time()   
        keys,values =  line.split(';')
        keys = int(keys)
        values = np.array(np.matrix(values)).ravel()
        
        y_train,array_id,array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec, max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec = init_vecs()
        array_id,array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec, max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec = get_musics_informations(array_id,array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec, max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec,values)
        
        dataframe,index = generate_input_dataset(array1,array2,tuples,loudness_vec,confidence_vec,loudness_max_vec, max_time_vec,loudness_start_vec,pitches_vec,segments_start_vec,segments_timbre_vec) 
        dataframe_out = generate_output_dataset(tags,output,keys,values,index,array_id)

        
        
        #dataframe = pd.read_csv('./../datacsv/%s.csv' % keys,index_col = ['ID','Tempo'])
        #dataframe_out = pd.read_csv('./../datacsv_out/%s.csv' % keys,index_col = ['ID','Tempo'])
        
        try:
            dataframe = to_np_array(dataframe)
            dataframe_out = to_np_array(dataframe_out)
        
        except:
            log_file = open('log_error_file_test6txt','a')
            log_file.write(str(keys))
            log_file.write('\n')
            log_file.close()
            continue
        y_train = []
        for i in range(len(dataframe_out)):
            y_train.append(dataframe_out[i][0])


        y_train = np.asarray(y_train).reshape((-1, 50))
         
        for c_mat_it in range(len(y_train)):
            y_pred = model.predict(dataframe)
            real_vec.append(np.asarray(y_train)[c_mat_it])
            pred_vec.append(np.asarray(y_pred)[c_mat_it])
            score = sklearn.metrics.roc_auc_score(np.asarray(y_train)[c_mat_it], np.asarray(y_pred)[c_mat_it])
            scores.append(score)
            
            
            
                
        
        if count%100==0:
            print (size,count,(time.time()-start_time),(str(sklearn.metrics.roc_auc_score(np.asarray(real_vec), np.asarray(pred_vec),average='micro'))),(str(sklearn.metrics.roc_auc_score(np.asarray(real_vec), np.asarray(pred_vec),average='samples'))))
            
            #print y_pred
            #print 'Auc_score: %s' % (str(auc_value))
            #model.save('./models/model_auc_%s_l_%s_e_%s_r_%s' % ('2_32',epochs_manual,epochs,it_epochs)) 
        count +=1
        
        #array_id.append([hdf5_getters.get_song_id(h5)])
        #array1.append([hdf5_getters.get_song_id(h5)] *len(confidence))
        #array2.append( range(len(confidence)))
    
    #print (sklearn.metrics.roc_auc_score(np.asarray(real_vec), np.asarray(pred_vec),average='macro'))
    result = open('./out/out_auc_%s_%s.txt' % (camadas,epochs),'w')
    result.write('AUC micro: ')
    result.write(str(sklearn.metrics.roc_auc_score(np.asarray(real_vec), np.asarray(pred_vec),average='micro')))
    result.write('\nAUC samples: ')
    result.write(str(sklearn.metrics.roc_auc_score(np.asarray(real_vec), np.asarray(pred_vec),average='samples')))
    result.close()

_test
_test 0 1.4498462677001953 0.30612244898 0.30612244898
_test 100 9.854448795318604 0.667817725169 0.682854229133
_test 200 26.73434090614319 0.638341093415 0.631991972545
_test 300 57.259369134902954 0.609131776256 0.598089835568
_test 400 125.06708145141602 0.571457427616 0.552529917325
_test 500 283.0502371788025 0.550484388808 0.531315198813
_test 600 555.1039144992828 0.551062340536 0.532486049501
_test 700 856.5468904972076 0.555874795739 0.53890832629
_test 800 1147.2136034965515 0.561948627019 0.547583987539
_test 900 1405.506820678711 0.568263531902 0.555653526873
_test 1000 1611.6290516853333 0.574203838891 0.562776941033
_test 1100 1765.668612241745 0.577824004782 0.567496690793
_test 1200 1895.8037152290344 0.581564224129 0.571760608308
_test 1300 1993.7796730995178 0.584188896223 0.575172611868
_test 1400 2076.9867973327637 0.586472516352 0.578046827466
_test 1500 2149.2479898929596 0.588950502147 0.58101083598
_test 1600 2211.754189491272 0.590975871716 0.58375483169

The prints are only for me to check faster without the need to open the log files. The real analysis was made using the log files.